# Yellow Taxi Analysis

## Downloading the data

In [1]:
! curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o yellow_tripdata_2023-01.parquet
! curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -o yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0   138M      0 --:--:-- --:--:-- --:--:--  138M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0   140M      0 --:--:-- --:--:-- --:--:--  140M


## Data Analysis

### Importing python Modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Importing Datasets

In [3]:
jan_df = pd.read_parquet('yellow_tripdata_2023-01.parquet')
feb_df = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [4]:
jan_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### Answering Homework Questions

#### Q1. Downloading the data
Read the data for January. How many columns are there?

In [5]:
jan_df.shape

(3066766, 19)

the january dataframe has 19 columns

#### Q2. Computing duration
What's the standard deviation of the trips duration in January?

In [6]:
jan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [7]:
def adding_duration_column(df):
  df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
  df["duration"] = df["duration"].dt.total_seconds() /60

In [8]:
#Adding Duration column as time delta in minutes between
#tpep_dropoff_datetime & tpep_pickup_datetime

adding_duration_column(jan_df)

In [9]:
jan_df["duration"].std()

42.59435124195458

standard deviation of the trips duration in January is 42.59 Minutes

#### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [10]:
def oulier_removal(df):
  df_copy = df.copy()
  df_copy = df_copy[
    (df_copy["duration"] <= 60)
                 & (df_copy["duration"] >= 1)
                 ]
  return df_copy

In [11]:
cleaned_jan_df = oulier_removal(jan_df)

In [12]:
len(cleaned_jan_df) / len(jan_df) * 100

98.1220282212598

The fraction of the records left after you dropped the outliers is 98%

#### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

In [13]:
features_columns = ["PULocationID", "DOLocationID"]
training_features = cleaned_jan_df[features_columns].astype(str)
training_targets = cleaned_jan_df["duration"]

In [14]:
training_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   PULocationID  object
 1   DOLocationID  object
dtypes: object(2)
memory usage: 68.9+ MB


In [15]:
one_hot_encoder = OneHotEncoder(handle_unknown = 'ignore')
transformed_features = one_hot_encoder.fit_transform(training_features)



In [16]:
transformed_features.shape

(3009173, 515)

In [17]:
lr_model = LinearRegression()
lr_model.fit(transformed_features, training_targets)

LinearRegression()

In [18]:
trainig_predictions = lr_model.predict(transformed_features)

In [19]:
trainig_rmse = mean_squared_error(training_targets, trainig_predictions, squared=False)

trainig_rmse

7.649261950799829

#### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [20]:
#Adding duration column
adding_duration_column(feb_df)

#Remove the Outliers
cleaned_feb_df = oulier_removal(feb_df)


#Extracting model features and target from testing data
test_features = cleaned_feb_df[features_columns].astype(str)
test_targets = cleaned_feb_df["duration"]

#One Hot Encode the Features
test_features = one_hot_encoder.transform(test_features)

#Make Testing Predictions
test_predictions = lr_model.predict(test_features)

#Calculate Testing RMSE
test_rmse = mean_squared_error(test_targets, test_predictions, squared=False)

test_rmse


7.81181792123828